In [1]:
#autoreload
%load_ext autoreload
%autoreload 2


In [1]:
# Crear una instancia del ambiente
import numpy as np
import matplotlib.pyplot as plt
from agente import AgenteQLearning
#from DQN import AgenteDQN
from ambiente import AmbienteDiezMil
from template import Validador
#from jugador import JugadorFromDQNPolicy
from IPython.display import clear_output
from tqdm import tqdm
from utils import RangeSnaper

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Inicialización del gráfico
def train(epochs, episodios, alpha, gamma, epsilon, rs, early_stop = 20, seed = 42):    
    seed = seed
    np.random.seed(seed)

    ambiente = AmbienteDiezMil(rs=rs)

    # Crear un agente de Q-learning
    agente = AgenteQLearning(ambiente, alpha=alpha, gamma=gamma, epsilon=epsilon)


    vals_normal = []
    best_validation = 30
    best_policy = None
    epochs_sin_mejora = 0
    # Entrenar al agente con un número de episodios
    for _ in range(epochs):
        agente.entrenar(episodios, verbose=False)
        if _ % 2 == 0:
            validador = Validador(AmbienteDiezMil(rs=rs))
            vals_normal.append(validador.validar_politica(agente.q_table2pol(), 300))
            if vals_normal[-1] < best_validation:
                best_validation = vals_normal[-1]
                best_policy = agente.q_table2pol()
                epochs_sin_mejora = 0
            else:
                epochs_sin_mejora += 1
                if epochs_sin_mejora == early_stop:
                    break
    
    return best_validation, best_policy, vals_normal

In [3]:
bv, bp, vn = train(100, 100, 0.1, 0.9, 0.1, RangeSnaper())

In [4]:
bv, bp

(21.32,
 array([[1, 0, 0, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 1]]))

In [39]:
RANGOS1 = [
    (0, 1), (1, 250), (250, 500),(500, 750), (750, 1000), (1000, 2000), (2000, 5000), (5000, 10000), (10000, int(1e10))
]

RANGOS2 = [
    (0, 1),(1, 50),(50, 100), (100, 150), (150, 200), (200, 250),
    (250, 350), (350, 500), (500, 750), (750, 1000), (1000, 2000),
    (2000, 5000), (5000, 10000), (10000, int(1e10))
]

RANGOS3 = [(0, 1), (1, 50), (50, 100), (100, 150), (150, 200), 
           (200, 250), (250, 350), (350, 500), (500, 750), (750, 1000), 
           (1000, 2000), (2000, 4000), (4000, 6000), (6000,8000),(8000,10000),(10000, int(1e10))]

RANGOS4 = [
    (0,1), (1, 250), (250, 500),(500, 750), (750, 1000),(1000, 5000), (5000, int(1e10))
    ]


In [40]:
#hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(params):
    alpha = params["alpha"]
    gamma = params["gamma"]
    epsilon = params["epsilon"]
    #reward = params["reward"]
    rango = params["rango"]

    best_validation, best_policy, _ = train(epochs=500, episodios=1000, alpha=alpha, gamma=gamma, epsilon=epsilon, rs=RangeSnaper(rango))
    
    return {
        "loss": best_validation,
        "status": STATUS_OK,
        "best_policy": best_policy
    }

space = {
    "alpha": hp.uniform("alpha", 0.0, 1.0),
    "gamma": hp.uniform("gamma", 0.0, 1.0),
    "epsilon": hp.uniform("epsilon", 0.01, 0.2),
    #"reward": hp.choice("reward", [0, 1, 2, 3]),
    "rango": hp.choice("rango", [RANGOS1, RANGOS2, RANGOS4])
}

trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=trials, verbose=1)
print(best)


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

In [ ]:
def getBestModelfromTrials(trials):
    valid_trial_list = [trial for trial in trials
                            if STATUS_OK == trial['result']['status']]
    losses = [ float(trial['result']['loss']) for trial in valid_trial_list]
    index_having_minumum_loss = np.argmin(losses)
    best_trial_obj = valid_trial_list[index_having_minumum_loss]
    return best_trial_obj['result']['best_policy']

best_policy = getBestModelfromTrials(trials)

In [ ]:
validador = Validador(AmbienteDiezMil(rs=RangeSnaper()))
(validador.validar_politica(best_policy, 300))
            

21.073333333333334

In [ ]:
bv, bp , _ =train(100, 1000, best["alpha"], best["gamma"], best["epsilon"], RangeSnaper())
validador = Validador(AmbienteDiezMil(rs=RangeSnaper()))
(validador.validar_politica(bp, 300)), bv 

(21.67, 21.14)

Parametros a definir:
Recompensa = {
    0: (1 si gana, 0 si suma, -1 si no suma),
    1:  (1 si suma, -1 si no suma),
    2: (puntos si suma, -puntos si no suma),
    3: (puntos/#tiros si suma, -puntos/#tiros si no suma),
}

Rango = RANGOS 1, RANGOS2, RANGOS3
Alpha = [0,1]
Gamma = [0,1]
Epsilon = [0,1]


In [4]:
from jugador import JugadorAleatorio
from template import Validador


val = Validador(ambiente)
val_count = 1000
avg = val.validar_politica(agente.q_table2pol(), val_count)
print(f"Validación DeepPurple: {avg}")

val_rand = Validador(ambiente)
jugador = JugadorAleatorio("random")
avg_rand = val_rand.validar_jugador(jugador, val_count)
print(f"Validación Random: {avg_rand}")

Validación DeepPurple: 25.066
Validación Random: 26.927


In [ ]:
%matplotlib inline
import time
# import pylab as pl
from IPython import display


ambiente = AmbienteDiezMil()

# Crear un agente de Q-learning
agente = AgenteQLearning(ambiente)

vals = []

for i, val in enumerate(agente.entrenar(episodios, verbose=True)):
    vals.append(val)
    plt.figure(figsize=(10, 5))
    plt.plot(vals)
    display.clear_output(wait=True)
    # display.display(pl.gcf())
    plt.show()